In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import datetime
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

res = requests.get('https://www.taifex.com.tw/cht/9/futuresQADetail')
soup = BeautifulSoup(res.text, 'html.parser')
name_a_elements = soup.find_all('td', headers='name_a') 
comp_list = [] #create list 

for e in name_a_elements:
    number = e.text.strip()
    if number.isdigit(): 
        comp_list.append(number)
comp_name = pd.DataFrame(comp_list, columns=['comp_name'])
comp_name_200 = comp_name[comp_name.index < 200]
count = len(comp_name_200 )
print(comp_name_200)
print('Count of companies:', count)

    comp_name
0        2330
1        2317
2        2454
3        2881
4        2308
..        ...
195      3030
196      2467
197      2645
198      4919
199      1736

[200 rows x 1 columns]
Count of companies: 200


In [8]:
def calculate_stock_metrics(sid,a,b, c, d):
    start = datetime.datetime.now() - datetime.timedelta(days=a)
    end = datetime.date.today()
    

    stock_yf = yf.download(sid+'.TW', start, end)
    

    stock_yf['SMA'] = stock_yf['Adj Close'].rolling(b).mean()
    stock_yf['STD'] = stock_yf['Adj Close'].rolling(b).std()


    stock_yf['upper'] = stock_yf['SMA'] + 1 * stock_yf['STD']
    stock_yf['lower'] = stock_yf['SMA'] - 1 * stock_yf['STD']
    

    stock_yf['1.5upper'] = stock_yf['SMA'] + 1.5 * stock_yf['STD']
    stock_yf['1.5lower'] = stock_yf['SMA'] - 1.5 * stock_yf['STD']
    
 
    stock_yf['EMA_short'] = stock_yf['Adj Close'].ewm(span=c, adjust=False).mean()
    stock_yf['EMA_long'] = stock_yf['Adj Close'].ewm(span=d, adjust=False).mean()
    
    btest=stock_yf.loc[:,['Adj Close','upper','lower','EMA_short','EMA_long']]
    btest['Entry']=(btest['EMA_short']>btest['lower']) & (btest['EMA_short'].shift(1)<btest['lower'])
    btest['Exit']=(btest['Adj Close']<btest['upper']) & (btest['Adj Close'].shift(1)>btest['upper'])

    return btest


In [9]:
def backtest_stock(btest):
    spread_side = None
    backtest_result = []
    price = None
    
    for idx, row in btest.iterrows():
        if spread_side is None:
            if row['Entry']:
                spread_side = "buy"
                price = row['Adj Close']
                return_stock = 0.0
                backtest_result.append([idx, return_stock, spread_side, price])
            
        elif spread_side == "buy":
            if row['Exit']:
                spread_side = "sell"
                return_stock = row['Adj Close'] / price - 1.0
                entry_price = row['Adj Close']
                backtest_result.append([idx, return_stock, spread_side, price])
                spread_side = None
    backtest_df = pd.DataFrame(backtest_result, columns=['Date', 'Return', 'Spread_Side', 'Price'])
    total_return = backtest_df['Return'].sum()*100
    return total_return

In [10]:
results = []
for company in comp_name_200['comp_name']:
    stock_yf_1 = calculate_stock_metrics(company,360,50,8,26)
    stock_yf_2 = calculate_stock_metrics(company,1000,50,8,26)
    total_return_1 = backtest_stock(stock_yf_1)
    total_return_2 = backtest_stock(stock_yf_2)
    total_return = round(total_return_1, 2)
    total_return = round(total_return_2, 2)
    if total_return_1 > 20 and total_return_2 > 20: 
        results.append((company, total_return_1,total_return_2))
        
Results=pd.DataFrame(results, columns=['comp_name','short-term Total Return in %','long-term Total Return in %'])
print(Results)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********